### Imported the required packages

In [1]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
np.random.seed(42)


C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Constructed the variational circuit

In [2]:
# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_qubits*4, nr_layers, 4))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)
        qml.RZ(params[i, j, 3], wires=i)
        
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 3])

    


In [3]:
params

tensor([[[ 1.5605, -0.4344,  2.0348,  4.7847],
         [-0.7356, -0.7356,  4.9612,  2.4110]],

        [[-1.4749,  1.7045, -1.4559, -1.4631],
         [ 0.7601, -6.0107, -5.4190, -1.7665]],

        [[-3.1819,  0.9872, -2.8526, -4.4369],
         [ 4.6045, -0.7093,  0.2121, -4.4760]],

        [[-1.7102,  0.3485, -3.6160,  1.1803],
         [-1.8870, -0.9164, -1.8903,  5.8191]],

        [[-0.0424, -3.3229,  2.5841, -3.8354],
         [ 0.6562, -6.1565, -4.1726,  0.6185]],

        [[ 2.3200,  0.5384, -0.3633, -0.9459],
         [-4.6449, -2.2615, -1.4471,  3.3210]],

        [[ 1.0795, -5.5388,  1.0181, -1.2098],
         [-2.1266,  1.9216,  3.2390,  2.9257]],

        [[-2.6365, -0.9714,  1.0407,  3.0648],
         [-1.5054, -0.5833, -3.4757, -3.7580]],

        [[ 2.5526,  4.2608, -0.2262,  3.1527],
         [ 1.1361, -2.0267,  1.1354,  4.8319]],

        [[-0.1126,  4.9155, -8.2302,  2.5821],
         [ 0.2735, -0.9394,  0.2883, -6.2441]],

        [[-0.6901,  1.1219,  4.6429, -1.

### We use the "default.qubit" device to perform the optimization

In [4]:
dev = qml.device("default.qubit", wires=4)


In [5]:
@qml.qnode(dev, interface="torch")
def circuit(params, A,input_1):
    
   
    
    
    qml.QubitStateVector(input_1, wires=[0,1,2,3])
        
  
        
        
    
    # repeatedly apply each layer in the circuit
    for j in range(nr_layers):
        layer(params, j)
    
    # measure the expectation value for each wire
    a=qml.expval(qml.Hermitian(A, wires=0))
    b=qml.expval(qml.Hermitian(A, wires=1))
    c=qml.expval(qml.Hermitian(A, wires=2))
    d=qml.expval(qml.Hermitian(A, wires=3))
    

    
    # returns the expectation of the input matrix A on the first qubit:      
    return a,b,c,d

### Prepare a random state-vector on 4 qubits

In [6]:
from qiskit import quantum_info as qf
rand_sv_1=np.array(qf.random_statevector(16))
rand_sv_2=np.array(qf.random_statevector(16))
rand_sv_3=np.array(qf.random_statevector(16))
rand_sv_4=np.array(qf.random_statevector(16))

rand_sv_f= np.array([rand_sv_1,rand_sv_2,rand_sv_3,rand_sv_4])


rand_sv_f

array([[ 0.20889538+0.08235256j, -0.03927539+0.16413401j,
         0.01705947+0.03113048j,  0.16858427+0.26197478j,
        -0.31613047+0.20626543j,  0.04069385+0.02418946j,
        -0.3536006 +0.14878745j,  0.07084692-0.15233711j,
        -0.01948252+0.23750527j, -0.22589945+0.16266235j,
         0.01636915-0.23158514j, -0.35253193+0.04063286j,
         0.07828291-0.29178605j,  0.01716371-0.16414925j,
        -0.06432171+0.14734933j,  0.18813903-0.09383527j],
       [ 0.03209248-0.23957826j,  0.13043321-0.127946j  ,
         0.26388014+0.18854133j,  0.01573879+0.01468668j,
         0.28077312-0.28359107j, -0.08439142-0.12695547j,
        -0.01416332-0.23236245j,  0.08652411+0.04140239j,
         0.0329493 +0.13733234j, -0.03226748-0.13910798j,
         0.0091103 +0.16859443j, -0.09566516+0.21953481j,
        -0.12679844+0.32075745j,  0.19197097+0.07725978j,
         0.26076317-0.27984902j, -0.34966845+0.00981328j],
       [-0.24780512+0.12790782j, -0.31394969+0.05104097j,
        -0.0

### Prepare the cost function and the optimisation routine

In [8]:

Paulis = Variable(torch.zeros([4, 2, 2], dtype=torch.complex128), requires_grad=False)

# Use Pauli-Z as the Hermitian operator
Paulis[0] = torch.tensor([[1, 0], [0, -1]])
Paulis[1] = torch.tensor([[1, 0], [0, -1]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])
Paulis[3] = torch.tensor([[1, 0], [0, -1]])


## Required o/ps
## Rows are the required output states
bloch_v=np.array([[0,1,0,1],
                  [0,0,1,1],
                  [1,0,1,0],
                  [1,1,0,0]])   
   
    

## Cost function
def cost_fn(params):
    f=0
    tt=np.zeros(4)
    for input_1 in range(4):
        a,b,c,d=circuit(params[4*input_1:(4*input_1)+4], Paulis[0],rand_sv_f[input_1])
        tt=[a,b,c,d]
        
        for k in range(4):
                     
            f+=torch.abs(tt[k] - bloch_v[input_1][k])
     
        
       
    return (f)



#set up the optimizer
opt = torch.optim.Adam([params], lr=0.001)

# number of steps in the optimization routine
steps = 1000

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits*4, nr_layers, 4))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))




Cost after 0 steps is 8.5038


C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\torch\autograd\__init__.py:147: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ..\aten\src\ATen\native\Copy.cpp:240.)
  Variable._execution_engine.run_backward(


Cost after 10 steps is 8.3036
Cost after 20 steps is 8.0826
Cost after 30 steps is 7.8637
Cost after 40 steps is 7.6593
Cost after 50 steps is 7.4567
Cost after 60 steps is 7.2544
Cost after 70 steps is 7.0828
Cost after 80 steps is 6.9044
Cost after 90 steps is 6.7516
Cost after 100 steps is 6.6033
Cost after 110 steps is 6.4614
Cost after 120 steps is 6.3238
Cost after 130 steps is 6.1948
Cost after 140 steps is 6.0670
Cost after 150 steps is 5.9462
Cost after 160 steps is 5.8271
Cost after 170 steps is 5.7134
Cost after 180 steps is 5.6137
Cost after 190 steps is 5.5153
Cost after 200 steps is 5.4208
Cost after 210 steps is 5.3297
Cost after 220 steps is 5.2388
Cost after 230 steps is 5.1499
Cost after 240 steps is 5.0782
Cost after 250 steps is 5.0064
Cost after 260 steps is 4.9379
Cost after 270 steps is 4.8683
Cost after 280 steps is 4.8006
Cost after 290 steps is 4.7332
Cost after 300 steps is 4.6669
Cost after 310 steps is 4.6010
Cost after 320 steps is 4.5353
Cost after 330 st

### Print the o/p results and compare with the desired o/p

In [12]:
output_bloch_v = [0,0,0,0]
for input_1 in range(4):
    output_bloch_v[input_1] = circuit(best_params[4*input_1:(4*input_1)+4], Paulis[0],rand_sv_f[input_1])
        


# o/p after training
print("Output after training :")
print(output_bloch_v)

# desired o/p
print("Desired o/p :")
print(bloch_v)


Output after training :
[tensor([-8.6769e-05,  8.7489e-01, -9.4963e-05,  8.2369e-01],
       dtype=torch.float64, grad_fn=<CatBackward>), tensor([-4.2521e-04,  1.5763e-04,  6.8000e-01,  6.9708e-01],
       dtype=torch.float64, grad_fn=<CatBackward>), tensor([ 7.6410e-01, -1.3571e-05,  8.8403e-01,  4.8339e-05],
       dtype=torch.float64, grad_fn=<CatBackward>), tensor([ 9.0984e-01,  6.7684e-01,  3.1843e-04, -3.9108e-04],
       dtype=torch.float64, grad_fn=<CatBackward>)]
Desired o/p :
[[0 1 0 1]
 [0 0 1 1]
 [1 0 1 0]
 [1 1 0 0]]
